# Hyperparameter optimization 


In [7]:
# Install needed packages
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time

In [8]:
# Import the data and prepare for further processing
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [9]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data.csv')
train_df = load_and_prepare_data('../../data/Final_data/train_df.csv')
test_df = load_and_prepare_data('../../data/Final_data/test_df.csv')

# Reset the index
df = df.reset_index()
train_df = train_df.reset_index()
test_df = test_df.reset_index()

#rename date and Day_ahead_price columns to ds and y
df = df.rename(columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})
train_df = train_df.rename(columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})
test_df = test_df.rename(columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})

## Parallelized version for Hyperparameter Search

In [10]:
import itertools
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time

# Define the hyperparameter grid
param_grid = {  
    'changepoint_prior_scale': [0.05, 0.1],
    'seasonality_prior_scale': [5],
    "holidays_prior_scale": [5],
    'seasonality_mode': ['additive', 'multiplicative'],
}

param_dist = {  
    'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.25, 0.5],
    'seasonality_prior_scale': [1, 5, 10],
    'holidays_prior_scale': [1, 5, 10],
    'seasonality_mode': ['additive', 'multiplicative'],
}


# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_dist.values())]

# Function to run Prophet with a specific parameter set
def evaluate_params(params):
    try:
        # Fit model with given params
        m = Prophet(**params).fit(df)
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days')
        df_p = performance_metrics(df_cv, rolling_window=0.1)

        # Calculate the average RMSE across all horizons
        avg_rmse = df_p['rmse'].mean()

        return params, avg_rmse
    except Exception as e:
        # Handle any exceptions that occur during evaluation
        print(f"Error occurred with parameters {params}: {e}")
        return params, float('inf'), pd.Timedelta(seconds=0)

# Start the timer
start_time = time.time()

# Use ThreadPoolExecutor to parallelize the evaluation
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(evaluate_params, all_params), total=len(all_params)))

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time:.2f} seconds")

# Extract the RMSEs, horizons, and corresponding params
rmses = [result[1] for result in results]
tuning_results = pd.DataFrame([result[0] for result in results])
tuning_results['average_rmse'] = rmses

# Output the tuning results
print(tuning_results)


  0%|          | 0/90 [00:00<?, ?it/s]20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:07 - cmdstanpy - INFO - Chain [1] start processing
20:33:09 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/19 [00:00<?, ?it/s]

20:33:09 - cmdstanpy - INFO - Chain [1] start processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/19 [00:00<?, ?it/s]

20:33:10 - cmdstanpy - INFO - Chain [1] done processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing
20:33:10 - cmdstanpy - INFO - Chain [1] done processing


...

  0%|          | 0/19 [00:00<?, ?it/s]

20:47:04 - cmdstanpy - INFO - Chain [1] done processing
20:47:05 - cmdstanpy - INFO - Chain [1] start processing
20:47:05 - cmdstanpy - INFO - Chain [1] start processing
20:47:05 - cmdstanpy - INFO - Chain [1] done processing
20:47:05 - cmdstanpy - INFO - Chain [1] done processing
20:47:06 - cmdstanpy - INFO - Chain [1] done processing
20:47:06 - cmdstanpy - INFO - Chain [1] done processing
20:47:07 - cmdstanpy - INFO - Chain [1] start processing
20:47:07 - cmdstanpy - INFO - Chain [1] done processing
20:47:07 - cmdstanpy - INFO - Chain [1] start processing
20:47:07 - cmdstanpy - INFO - Chain [1] done processing
 88%|████████▊ | 79/90 [14:02<01:30,  8.27s/it]20:47:07 - cmdstanpy - INFO - Chain [1] start processing
20:47:07 - cmdstanpy - INFO - Chain [1] done processing
20:47:07 - cmdstanpy - INFO - Chain [1] done processing
20:47:08 - cmdstanpy - INFO - Chain [1] start processing
20:47:08 - cmdstanpy - INFO - Chain [1] start processing
20:47:09 - cmdstanpy - INFO - Chain [1] start proc

  0%|          | 0/19 [00:00<?, ?it/s]

 91%|█████████ | 82/90 [14:08<00:31,  3.92s/it]20:47:13 - cmdstanpy - INFO - Chain [1] done processing
20:47:13 - cmdstanpy - INFO - Chain [1] start processing
20:47:14 - cmdstanpy - INFO - Chain [1] done processing
20:47:14 - cmdstanpy - INFO - Chain [1] start processing
20:47:14 - cmdstanpy - INFO - Chain [1] start processing
20:47:14 - cmdstanpy - INFO - Chain [1] done processing
20:47:14 - cmdstanpy - INFO - Chain [1] done processing
20:47:14 - cmdstanpy - INFO - Chain [1] done processing
20:47:15 - cmdstanpy - INFO - Chain [1] start processing
20:47:15 - cmdstanpy - INFO - Chain [1] done processing
20:47:15 - cmdstanpy - INFO - Chain [1] done processing
20:47:15 - cmdstanpy - INFO - Chain [1] start processing
20:47:16 - cmdstanpy - INFO - Chain [1] start processing
 92%|█████████▏| 83/90 [14:10<00:24,  3.45s/it]20:47:16 - cmdstanpy - INFO - Chain [1] start processing
20:47:16 - cmdstanpy - INFO - Chain [1] start processing
20:47:16 - cmdstanpy - INFO - Chain [1] start processing
2

Total execution time: 912.19 seconds
    changepoint_prior_scale  seasonality_prior_scale  holidays_prior_scale  \
0                      0.01                        1                     1   
1                      0.01                        1                     1   
2                      0.01                        1                     5   
3                      0.01                        1                     5   
4                      0.01                        1                    10   
..                      ...                      ...                   ...   
85                     0.50                       10                     1   
86                     0.50                       10                     5   
87                     0.50                       10                     5   
88                     0.50                       10                    10   
89                     0.50                       10                    10   

   seasonality_mode  avera

In [11]:
# Python
best_params = all_params[np.argmin(rmses)]
print(best_params)


{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 1, 'holidays_prior_scale': 1, 'seasonality_mode': 'additive'}
